# Predict - Alexnet

In [1]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from utilities import load_images, get_multi_hot_labels
from alexnet_model import alexnet_model_fn
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# Load all test images and labels
test_df = pd.read_csv('data/test.csv')

test_path_list = test_df['imagePath']

In [3]:
def predict(test_data):
    multilabel_classifier = tf.estimator.Estimator(
            model_fn=alexnet_model_fn, model_dir="model/multilabel_alexnet_model")
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": test_data},
            shuffle=False)
    predictions = multilabel_classifier.predict(input_fn=test_input_fn)
    y_predicted = np.array(list(p['classes'] for p in predictions))
    label_id = []
    image_id = list(range(1, y_predicted.shape[0]+1))
    for i in range(y_predicted.shape[0]):
        label_array = np.where(y_predicted[i]==1)[0]
        label_array += 1
        label_string = ""
        for j in range(label_array.shape[0]):
            label_string += " " + str(label_array[j])
        label_string = label_string[1:]
        label_id.append(label_string)
    submission = pd.DataFrame({'image_id':image_id, 'label_id':label_id})
    return submission

In [4]:
test_data = load_images(test_path_list[:20000])
submission_1 = predict(test_data)

del test_data
gc.collect()

test_data = load_images(test_path_list[20000:])
submission_2 = predict(test_data)

submission = submission_1.append(submission_2, ignore_index=True)

Loading images: 0
Loading images: 1000
Loading images: 2000
Loading images: 3000
Loading images: 4000
Loading images: 5000
Loading images: 6000
Loading images: 7000
Loading images: 8000
Loading images: 9000
Loading images: 10000
Loading images: 11000
Loading images: 12000
Loading images: 13000
Loading images: 14000
Loading images: 15000
Loading images: 16000
Loading images: 17000
Loading images: 18000
Loading images: 19000
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/multilabel_alexnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002371B53CAC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_mast

In [5]:
submission.to_csv('submission_0.499_localtest.csv', index = False)